In [1]:
import cv2
import numpy
import matplotlib.pyplot as plt
from glob import glob
from os import path

In [2]:
from skimage.color import *
from skimage.util import *

In [3]:
def show_image(my_image):
    fig, ax = plt.subplots(dpi=120)
    ax.imshow(my_image)
    ax.axis('off')
    plt.show()

In [4]:
def load_image(image_file):
    img_bgr = cv2.imread(image_file)
    my_image = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB) 
    return my_image

In [5]:
def is_low_contrast(obj_image, fraction_threshold=0.05, lower_percentile=1,
                    upper_percentile=99, method='linear'):
    dlimits = dtype_limits(obj_image, clip_negative=False)
    limits = numpy.percentile(obj_image, [lower_percentile, upper_percentile])
    
    ratio = (limits[1] - limits[0]) / (dlimits[1] - dlimits[0])

    return ratio < fraction_threshold, ratio

In [6]:
def is_low_contrast_image(obj_image, fraction_threshold=0.2, lower_percentile=10, upper_percentile=90):
    low_contrast = False

    low_contrast = is_low_contrast(obj_image, 
                                   fraction_threshold=fraction_threshold, 
                                   lower_percentile=lower_percentile, 
                                   upper_percentile=upper_percentile)
    return low_contrast

In [7]:
def output(fraction_threshold, lower_percentile, upper_percentile):
    global obj_images
    
    low_contrast_images = []
    low_contrast_ratios = []

    for obj_name in obj_images:
        obj = obj_images[obj_name]

        low_contrast, ratio = is_low_contrast_image(obj, fraction_threshold=fraction_threshold,
                                                    lower_percentile=lower_percentile, upper_percentile=upper_percentile)
        low_contrast_ratios.append(ratio)

        if not low_contrast:
            low_contrast_images.append(obj_name)

    if len(low_contrast_images) > 0:
        print(len(low_contrast_images), len(low_contrast_images) / len(image_files))

    print(numpy.mean(low_contrast_ratios), numpy.std(low_contrast_ratios))

<h3>環境設置</h3>

In [8]:
image_folder = 'D:/Projects/ComputerVision/AutoEncoder/Fish/images/objects/faint'
image_files = [ f[len(image_folder) + 1:] for f in sorted(glob('%s/F*.jpg' % (image_folder)))]

image_files[:5]

['F000234-00001.jpg',
 'F000235-00001.jpg',
 'F000236-00000.jpg',
 'F000237-00001.jpg',
 'F000238-00001.jpg']

<h3>轉換資料格式</h3>

In [9]:
obj_images = dict()

for image_file in image_files:
    _image_file = '%s/%s' % (image_folder, image_file)

    my_image = load_image(_image_file)
    _image = rgb2gray(numpy.asanyarray(my_image))
    
    obj_images[image_file] = _image

<h3>設定基礎閥值，計算模糊影像個數</h3>

In [10]:
output(fraction_threshold=0.1, lower_percentile=5, upper_percentile=95)

97 0.8220338983050848
0.12800642198404782 0.026211590387338134


<h3>提高閥值，亦會增加模糊影像個數</h3>

In [11]:
output(fraction_threshold=0.15, lower_percentile=5, upper_percentile=95)

27 0.2288135593220339
0.12800642198404782 0.026211590387338134


<h3>提高閥值，亦會增加模糊影像個數</h3>

In [12]:
output(fraction_threshold=0.20, lower_percentile=5, upper_percentile=95)

0.12800642198404782 0.026211590387338134


<h3>調整低區間百分位，會增加模糊影像個數</h3>

In [13]:
output(fraction_threshold=0.20, lower_percentile=15, upper_percentile=95)

0.1180384874543037 0.030365854567004184


<h3>調整高區間百分位，會增加模糊影像個數</h3>

In [14]:
output(fraction_threshold=0.20, lower_percentile=5, upper_percentile=85)

0.09466282685277495 0.019327676940458637
